# 🕵️ ReAct Agent: Jak AI używa narzędzi?

Modele językowe nie umieją liczyć (tokenizacja!) i nie mają dostępu do Google.
Żeby to naprawić, stosujemy wzorzec **ReAct**.

Nie jest to żadna nowa sieć neuronowa. To po prostu sprytny **Prompt Engineering + Pętla w Pythonie**.

**Algorytm:**
1.  Wstrzykujemy do Promptu instrukcję: *"Masz dostęp do narzędzi: [Kalkulator, Wiki]. Używaj formatu: Thought -> Action -> Observation"*.
2.  Wysyłamy zapytanie do LLM.
3.  Jeśli LLM wygeneruje tekst `Action: Kalkulator`, nasz skrypt w Pythonie to przechwytuje (Regex).
4.  Uruchamiamy prawdziwy kalkulator `eval()`.
5.  Doklejamy wynik (`Observation: 42`) do historii rozmowy i wysyłamy z powrotem do LLM.
6.  LLM widzi wynik i kontynuuje myślenie.

Zbudujemy "Mini-LangChain" od zera.

In [1]:
import re

# 1. DEFINIUJEMY NARZĘDZIA (Tools)
# To są funkcje Pythonowe, których Agent będzie mógł użyć.

def wikipedia_search(query):
    # Symulacja wyszukiwarki (Mock)
    # W prawdziwym życiu tu byłoby zapytanie do API Wikipedii
    db = {
        "Python": "Python to język programowania stworzony przez Guido van Rossuma.",
        "Einstein": "Albert Einstein urodził się w 1879 roku.",
        "Warszawa": "Warszawa to stolica Polski, populacja ok. 1.8 mln."
    }
    return db.get(query, "Nie znaleziono informacji.")

def calculator(expression):
    # Prosty kalkulator
    try:
        return str(eval(expression))
    except:
        return "Błąd obliczeń."

# Rejestr narzędzi
tools = {
    "SEARCH": wikipedia_search,
    "CALC": calculator
}

print("Narzędzia gotowe: SEARCH, CALC.")

Narzędzia gotowe: SEARCH, CALC.


## Symulacja LLM (Mock)

Ponieważ nie chcemy tutaj wymagać klucza OpenAI API (płatnego), stworzymy klasę `FakeLLM`, która **udaje**, że jest GPT-4.
Będzie miała zaszyte "scenariusze" odpowiedzi w formacie ReAct.

Dzięki temu zobaczysz dokładnie, jak wygląda tekst przesyłany tam i z powrotem, bez wydawania dolarów.

In [2]:
class FakeGPT:
    def __init__(self):
        self.step = 0
        
    def generate(self, prompt):
        """
        Ta funkcja udaje GPT-4. W prawdziwym kodzie tutaj byłoby:
        response = openai.ChatCompletion.create(...)
        """
        print(f"\n--- [LLM MYŚLI...] ---\n(Otrzymał prompt o długości {len(prompt)} znaków)")
        
        # Scenariusz: Pytanie "Ile lat miałby Einstein dzisiaj (2025)?"
        # Krok 1: Model chce sprawdzić, kiedy urodził się Einstein.
        if "Action:" not in prompt and self.step == 0:
            self.step += 1
            return "Thought: Muszę sprawdzić datę urodzenia Einsteina.\nAction: SEARCH[Einstein]"
        
        # Krok 2: Model dostał datę (1879) i teraz chce policzyć wiek.
        elif "Observation: Albert Einstein urodził się w 1879 roku." in prompt and self.step == 1:
            self.step += 1
            return "Thought: Wiem, że urodził się w 1879. Mamy rok 2025. Muszę policzyć różnicę.\nAction: CALC[2025-1879]"
        
        # Krok 3: Model dostał wynik obliczeń i kończy zadanie.
        elif "Observation: 146" in prompt:
            return "Thought: Mam już wynik.\nFinal Answer: Albert Einstein miałby dzisiaj 146 lat."
            
        else:
            return "Error: Nie wiem co robić."

llm = FakeGPT()
print("FakeGPT gotowy do symulacji.")

FakeGPT gotowy do symulacji.


## Pętla Agenta (The Loop)

To jest serce LangChaina.
1.  Wstawiamy pytanie do Promptu.
2.  **While True:**
    *   Zapytaj LLM.
    *   Czy w odpowiedzi jest `Final Answer`? -> Koniec.
    *   Czy w odpowiedzi jest `Action: NARZĘDZIE[WSAD]`?
    *   Jeśli tak -> Uruchom funkcję Python, weź wynik.
    *   Doklej `Observation: WYNIK` do historii.

In [3]:
# System Prompt (Instrukcja dla modelu)
SYSTEM_PROMPT = """
Jesteś inteligentnym agentem. Masz dostęp do narzędzi:
- SEARCH[query]: Wyszukuje w Wikipedii.
- CALC[expression]: Liczy wyrażenia matematyczne.

Używaj formatu:
Question: Pytanie użytkownika
Thought: Twój proces myślowy
Action: NARZĘDZIE[argument]
Observation: Wynik z narzędzia
... (powtórz Thought/Action/Observation)
Final Answer: Ostateczna odpowiedź
"""

def run_agent(question):
    # Historia rozmowy zaczyna się od instrukcji i pytania
    history = SYSTEM_PROMPT + f"\nQuestion: {question}\n"
    
    for i in range(5): # Max 5 kroków (zabezpieczenie przed pętlą)
        # 1. Zapytaj Model
        response = llm.generate(history)
        
        # Dodaj odpowiedź modelu do historii
        history += response + "\n"
        print(f"🤖 AI: {response}")
        
        # 2. Sprawdź, czy koniec
        if "Final Answer:" in response:
            return response.split("Final Answer:")[1].strip()
        
        # 3. Parsowanie Akcji (Regex)
        # Szukamy wzorca: Action: NAZWA[argument]
        match = re.search(r"Action: (\w+)\[(.*?)\]", response)
        
        if match:
            tool_name = match.group(1)
            tool_input = match.group(2)
            
            # 4. Wykonanie Narzędzia
            if tool_name in tools:
                print(f"⚙️ SYSTEM: Uruchamiam {tool_name} z argumentem '{tool_input}'...")
                tool_result = tools[tool_name](tool_input)
                
                # 5. Doklejenie Obserwacji
                observation = f"Observation: {tool_result}"
                history += observation + "\n"
                print(f"👀 SYSTEM: {observation}")
            else:
                history += "Observation: Nie ma takiego narzędzia.\n"
        else:
            # Model coś wymamrotał bez akcji, kończymy
            break
            
    return "Nie udało się znaleźć odpowiedzi."

# URUCHAMIAMY!
pytanie = "Ile lat miałby Einstein dzisiaj (zakładając rok 2025)?"
wynik = run_agent(pytanie)

print("-" * 30)
print(f"💡 WYNIK KOŃCOWY: {wynik}")


--- [LLM MYŚLI...] ---
(Otrzymał prompt o długości 423 znaków)
🤖 AI: Error: Nie wiem co robić.
------------------------------
💡 WYNIK KOŃCOWY: Nie udało się znaleźć odpowiedzi.


## 🧠 Podsumowanie: Prompt jako Program

To, co właśnie zobaczyłeś, to mechanizm działania **LangChain** i **AutoGPT**.

Zauważ, co się stało:
1.  Model sam "zorientował się", że nie zna wieku Einsteina.
2.  Zdecydował użyć `SEARCH`.
3.  Dostał datę, ale nie umiał odjąć w pamięci (lub wolał nie ryzykować).
4.  Zdecydował użyć `CALC`.
5.  Dostał wynik i sformułował odpowiedź.

**To jest Inżynieria Systemów AI.**
Nie trenowaliśmy tu żadnej sieci. Zbudowaliśmy **środowisko (pętlę)**, w którym model może działać autonomicznie.
W prawdziwym świecie podmieniasz `FakeGPT` na `openai.ChatCompletion`, a `tools` na Google Search API i masz gotowego asystenta.